In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import numpy as np
from math import pi, cos

Функция, убирающая html синтаксис.

In [ ]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

getLinks
Функция, на вход которой будут подаваться округа Москвы, а она будет возвращать номера объявлений о продаже квартир из этого округа.

In [ ]:
def getLinks(main_url): # ссылка со страницами
    links = []
    for page in range(30):
        flats_page = requests.get(main_url.format(page))
        flats_page = flats_page.content
        flats_page = BeautifulSoup(flats_page, 'lxml')
        flats_urls = flats_page.findAll('a', attrs = {'class':"serp-item__card-link link"})
        flats_urls = re.split('http://www.cian.ru/sale/flat/|/"',str(flats_urls))
        for link in flats_urls:
            if link.isdigit():
                links.append(link)
    return links;

getRoom
Функция, возвращающая число квартир. Возвращает 0, если квартира - студия, и 9, если квартира многокомнатная.

In [ ]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'многокомн' in i: # дополнительно проверили на многокомнатность
            return 9
        if 'комн' in i:
            break
        else:
            room_number += i
    if room_number.find('студия')!=-1: # и на студию
        room_number = 0
    else:
        room_number = int("".join(room_number.split()))
    return room_number

getCoords
Функция, возвращающая долготу и широту.

In [ ]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

getDist
Функция, возвращающая долготу и широту, а также расстояние до центра Москвы в километрах. Использует сложнейшие формулы математического анализа.

In [ ]:
def getDist(flat_page):
    shirota, dolgota = getCoords(flat_page)
    y = (2*pi*6371)/360*(shirota-55.75370903771494) # расстояние в километрах по широте
    x = (2*pi*6371*cos(shirota*pi/180))/360*(dolgota-37.61981338262558) # расстояние в километрах по долготе
    dist = (x*x+y*y)**0.5 # формула Пифагора для полученных длин
    return dolgota, shirota, dist

getMetro
Функция, возврщающая время до метро в минутах и 1, если время указано для пешей прогулки, 0 иначе. Nan и 1 если время до метро не указано.

In [ ]:
def getMetro(flat_page):
    if str(flat_page).find('<span class="object_item_metro_comment">')==-1: # у некоторых квартир время до метро не указано
        return np.nan, 1
    metro = flat_page.find('span',attrs={'class':'object_item_metro_comment'})
    metro = re.split('\n', str(metro))
    time = int(metro[1].replace(' ','')) # берем время
    Walk = 0
    for i in metro:
        if i.replace(' ','')=='пешком': # и ищем желанное слово "пешком"
            Walk = 1
    return time, Walk

getPrice
Функция, возвращающая цену квартиры в рублях.

In [ ]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

getTable
Фунция, обрабатывающая стандартную табличку, описывающую квартиру. Возвращает (в порядке вывода):
1, если дом кирпичный/монолит/жб. 0 иначе.
1, если квартира в новостройке. 0 иначе.
Этаж квартиры и количество этажей всего. Nan, если не указано.
Общую площадь, жилую площадь и площадь кухни. Nan, если не указано.
1 при наличии балкона или лоджии, 0 иначе.
1 при наличии лифта, 0 иначе.
1 при наличии телефона в квартире, 0 иначе.
Словесное описание вида из окна.

In [ ]:
def getTable(flat_page):
    table = flat_page.find('table', attrs={'class':'object_descr_props flat sale'})
    s = html_stripper(table).split('\n')
    a = []
    for i in s:
        if i!='':
            a.append(i) # после этих операций у нас в (а) будут лежать слова в порядке их следования
        
    Phone = 0 # потому что графа может отсутствовать в табличке совсем
    for i in range(len(a)):
        if a[i]=='Тип дома:':
            j=i+1
            Brick = 0
            while a[j]!='Тип продажи:': # перебираем все слова до следующей строки таблички
                c = a[j].replace(' ','').replace(',','').replace('.','')
                if c=='вторичка':
                    New = 0
                if c=='новостройка':
                    New = 1
                if c=='панельный' or c=='монолитно-кирпичный' or c=='монолитный' or c=='блочный' or c=='кирпичный' or c=='сталинский':
                    Brick = 1 # сталинский - это же стиль вообще, намешали всего. В общем, брик=1 - дом "крепкий"
                j=j+1

        if a[i]=='Этаж:':
            if a[i+1]=='—' or a[i+1]=='–': # на случай, если этажи не указаны
                Floor = np.nan
                Nfloors = np.nan
            else:
                help_s = a[i+1].replace(' ','').split('\xa0')
                if len(help_s)==2 and help_s[1]=='': # может быть не указана этажность дома
                    Floor = int(help_s[0])
                    Nfloors = np.nan
                elif len(help_s)==3 and help_s[1]=='/':
                    Floor = int(help_s[0])
                    Nfloors = int(help_s[2])
                else:
                    Floor = np.nan
                    Nfloors = np.nan

        if a[i]=='Общая площадь:':
            if a[i+1]=='—' or a[i+1]=='–': # это разные "тире" на самом деле
                AllArea = np.nan
            else:
                AllArea = float(a[i+1][:(a[i+1].find('x')-2)].replace(',','.'))

        if a[i]=='Жилая площадь:':
            if a[i+1]=='—' or a[i+1]=='–':
                LivingArea = np.nan
            else:
                LivingArea = float(a[i+1][:(a[i+1].find('x')-2)].replace(',','.'))

        if a[i]=='Площадь кухни:':
            if a[i+1]=='—' or a[i+1]=='–':
                KitchenArea = np.nan
            else:
                KitchenArea = float(a[i+1][:(a[i+1].find('x')-2)].replace(',','.'))

        if a[i]=='Балкон:':
            if a[i+1]=='—' or a[i+1]=='–' or a[i+1]=='нет':
                Balcony = 0
            else:
                Balcony = 1

        if a[i]=='Лифт:':
            if a[i+1]=='—' or a[i+1]=='–' or a[i+1]=='нет':
                Elevator = 0
            else:
                Elevator = 1

        if a[i]=='Телефон:':
            if a[i+1]=='—' or a[i+1]=='–' or a[i+1]=='нет':
                Phone = 0
            else:
                Phone = 1
                
        if a[i]=='Вид из окна:': # вид из окна важен, я считаю
            View = a[i+1]

    return Brick, New, Floor, Nfloors, AllArea, LivingArea, KitchenArea, Balcony, Elevator, Phone, View

Включаем в нашу базу все округа, за исключением Зеленоградского, Новомосковского и Троицкого.

In [ ]:
okruga = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B1%5D=24&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B1%5D=40&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B1%5D=57&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B1%5D=73&district%5B10%5D=82&district%5B11%5D=83&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B1%5D=85&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B1%5D=101&district%5B10%5D=110&district%5B11%5D=111&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B1%5D=113&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
,'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1']

In [ ]:
data = pd.DataFrame(columns=['N', 'Okrug', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp','Dolgota',
'Shirota','Dist','Metrdist','Walk','Brick','Tel','Bal','Floor','Nfloors','New','View','Elevator'])

okr = -1

for i in okruga:
    okr = okr+1 # создаем категориальный признак для округа
    links = getLinks(i) # считываем все линки для данного округа
    print('Got links from %d'%okr)
    for j in range(len(links)):
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[j]) + '/'
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')
         # ищем наши признаки
        N = links[j]
        Rooms = getRoom(flat_page)
        Price = getPrice(flat_page)
        Dolgota, Shirota, Dist = getDist(flat_page)
        Metrdist, Walk = getMetro(flat_page)
        Brick, New, Floor, Nfloors, Totsp, Livesp, Kitsp, Bal, Elevator, Tel, View = getTable(flat_page)
         # добавляем их в базу
        to_append = {'N':N, 'Okrug':okr, 'Rooms':Rooms, 'Price':Price, 'Totsp':Totsp, 'Livesp':Livesp,
                        'Kitsp':Kitsp,'Dolgota':Dolgota,'Shirota':Shirota,'Dist':Dist,
                        'Metrdist':Metrdist,'Walk':Walk,'Brick':Brick,'Tel':Tel,'Bal':Bal,
                        'Floor':Floor,'Nfloors':Nfloors,'New':New,'View':View,'Elevator':Elevator}
        data = data.append(to_append, ignore_index=True)
        if j%10==0:
            print('Done: %d from %d'%(j,okr)) # отслеживаем работу

In [ ]:
CIAN = data.drop_duplicates(['N']) # на сиане все устроено как-то так, что попадаются дубликаты. Избавляемся от них.

In [ ]:
CIAN.to_csv('cian.csv',index=False) # сохраняем добытую дату